In [1]:
from keras.datasets import mnist
from keras import models
from keras import layers
from keras import backend as K
import keras
import numpy as np

Using TensorFlow backend.
/Users/panda/opt/anaconda3/envs/Keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/panda/opt/anaconda3/envs/Keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/panda/opt/anaconda3/envs/Keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/pand

In [3]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images/255.

dim1=train_images.shape[0]
dim2=train_images.shape[1]
dim3=train_images.shape[2]
train_images = train_images.reshape((dim1,dim2,dim3,1))

11493376/11490434 [==============================] - 2s 0us/step


In [6]:
input_shape=(28,28,1)
dim=2

In [4]:
encoder_input=layers.Input(shape=input_shape)
x=encoder_input
x=layers.Conv2D(16,3,padding='same',strides=(2,2))(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(8,3,padding='same',strides=(2,2))(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(8, 3, padding="same")(x)

shape_before_flattening = K.int_shape(x)[1:]

x=layers.Flatten()(x)

z_mean = layers.Dense(dim)(x)
z_log_var = layers.Dense(dim)(x)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], dim), mean=0. , stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z=layers.Lambda(sampling, name='z')([z_mean, z_log_var])

#decoder
decoder_input = layers.Input(shape=(dim,), name='decoder_input')
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(16,3,padding='same',strides=(2,2))(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(8,3,padding='same',strides=(2,2))(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(3,3, padding='same')(x)
x = layers.Activation('sigmoid')(x)

decoder_output = x
encoder = models.Model(encoder_input, z)
decoder = models.Model(decoder_input, decoder_output)
output = decoder(z)
vae = models.Model(encoder_input, output)
vae.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 14, 14, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 7, 8)      1160        leaky_re_lu_1[0][0]              
____________________________________________________________________________________________

In [5]:
def vae_r_loss(y_true, y_pred):
    r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
    return 5000 * r_loss

def vae_kl_loss(y_true, y_pred):
    kl_loss =  -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = 1)
    return kl_loss

def vae_loss(y_true, y_pred):
    r_loss = vae_r_loss(y_true, y_pred)
    kl_loss = vae_kl_loss(y_true, y_pred)
    return  r_loss + kl_loss

vae.compile(optimizer=keras.optimizers.Adam(lr=0.0005), loss = vae_loss,  metrics = [vae_r_loss, vae_kl_loss])

NameError: name 'vae' is not defined

In [6]:
vae.fit(train_images,train_images, epochs=3)

W0719 09:29:11.017223  6152 deprecation_wrapper.py:119] From C:\Users\유정훈\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/3
60000/60000 [==============================] - 64s 1ms/step - loss: 329.8483 - vae_r_loss: 325.2258 - vae_kl_loss: 4.6227
Epoch 2/3
60000/60000 [==============================] - 66s 1ms/step - loss: 270.1686 - vae_r_loss: 267.3039 - vae_kl_loss: 2.8650
Epoch 3/3
60000/60000 [==============================] - 67s 1ms/step - loss: 259.7469 - vae_r_loss: 256.8243 - vae_kl_loss: 2.9225


In [2]:
import matplotlib.pyplot as plt
n_to_show = 30

znew = np.random.normal(size = (n_to_show,dim))

reconst = decoder.predict(np.array(znew))

fig = plt.figure(figsize=(12, 15))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
for i in range(n_to_show):
    ax = fig.add_subplot(6, 5, i+1)
    ax.imshow(reconst[i, :,:,:])
    ax.axis('off')

plt.show()

NameError: name 'dim' is not defined